<a href="https://colab.research.google.com/github/alfhi24/FinalProjectDTI/blob/main/Project_DTI_Machine_Learning_Method_MLP_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import Library**


In [59]:
! pip3 install Sastrawi

In [60]:
! pip3 install imblearn

In [61]:
!pip install mlxtend

In [62]:
pip install scikit-plot

Note: you may need to restart the kernel to use updated packages.


In [63]:
# Library for dataset
import pandas as pd
import numpy as np

# Library for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Library for preprocessing
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder

#Library for classification
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

#Library for Evaluation
from sklearn import metrics
import time
from sklearn.metrics import classification_report, accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from scikitplot.metrics import plot_confusion_matrix as pcm



In [64]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### **Dataset**


In [65]:
df = pd.read_csv('https://raw.githubusercontent.com/asthala/racism-detection/master/datasetfix.csv')
df.head()

,tweets,label
0,Trus nih buat yg nanya kenapa gak dibangun di ...,Non_R
1,PLAY & Let The Fun Begin 😱\nLike + RT + Click ...,Non_R
2,Papua adalah bagian dari NKRI\n#DukungOtsusPap...,Non_R
3,Akar konflik harus dibicarakan.\nDan dicari ti...,Non_R
4,"""Saya minta kita semua jadi orang Papua. Artin...",Non_R


In [66]:
count_label =df.label.value_counts().to_frame()
px.bar(data_frame=count_label, x=count_label.index , y='label')

### **Preprocessing**

#### Data Cleaning

In [67]:
def clean(data):
  tweets = []
  for tweet in data:
      tweet = re.sub(r"'(?:\@|https?\://)\S+", "", tweet)
      tweet = re.sub('\n', '', tweet)
      tweet = re.sub('rt', '', tweet)
      tweet = re.sub("[^a-zA-Z^']", " ", tweet)
      tweet = re.sub(" {2,}", " ", tweet)
      tweet = tweet.strip()
      tweets.append(tweet)
  return tweets

In [68]:
df_clean = df.copy()
df_clean['tweets']= clean(df['tweets'])
df_clean.head()

,tweets,label
0,Trus nih buat yg nanya kenapa gak dibangun di ...,Non_R
1,PLAY Let The Fun Begin Like RT Click https www...,Non_R
2,Papua adalah bagian dari NKRI DukungOtsusPapua...,Non_R
3,Akar konflik harus dibicarakan Dan dicari titi...,Non_R
4,Saya minta kita semua jadi orang Papua Ainya k...,Non_R


#### Case Folding


In [69]:
def case_fold(data):
    return data.str.lower()

In [70]:
df_clean['tweets'] = case_fold(df_clean['tweets'])
df_clean.head()

,tweets,label
0,trus nih buat yg nanya kenapa gak dibangun di ...,Non_R
1,play let the fun begin like rt click https www...,Non_R
2,papua adalah bagian dari nkri dukungotsuspapua...,Non_R
3,akar konflik harus dibicarakan dan dicari titi...,Non_R
4,saya minta kita semua jadi orang papua ainya k...,Non_R


#### Tokenization


In [71]:
def token(data):
  return data.apply(nltk.word_tokenize)

In [72]:
df_clean['tweets'] = token(df_clean['tweets'])
df_clean.head()

,tweets,label
0,"[trus, nih, buat, yg, nanya, kenapa, gak, diba...",Non_R
1,"[play, let, the, fun, begin, like, rt, click, ...",Non_R
2,"[papua, adalah, bagian, dari, nkri, dukungotsu...",Non_R
3,"[akar, konflik, harus, dibicarakan, dan, dicar...",Non_R
4,"[saya, minta, kita, semua, jadi, orang, papua,...",Non_R


#### Stopword


In [73]:
def stop_words(data) :
  stop_words = set(stopwords.words('indonesian'))
  return data.apply(lambda x: [item for item in x if item not in stop_words])

In [74]:
df_clean['tweets'] = stop_words(df_clean['tweets'])
df_clean.head()

,tweets,label
0,"[trus, nih, yg, nanya, gak, dibangun, papua, a...",Non_R
1,"[play, let, the, fun, begin, like, rt, click, ...",Non_R
2,"[papua, nkri, dukungotsuspapuamaju]",Non_R
3,"[akar, konflik, dibicarakan, dicari, titik, te...",Non_R
4,"[orang, papua, ainya, kulit, hitam, kulit, put...",Non_R


#### Stemming


In [75]:
def stem(data):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  return data.apply(lambda x: [ stemmer.stem(item) for item in x])

In [ ]:
df_clean['tweets'] = stem(df_clean['tweets'])
df_clean.head()

#### Racism Tweet Word Cloud

In [ ]:
word = df_clean.tweets[df_clean['label']=='R'].astype('string')
word.head()
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate_from_text(' '.join(word))

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#### TF-IDF


In [ ]:
def tf_idf(data):
  vectorizer = TfidfVectorizer()
  return vectorizer, vectorizer.fit_transform(data)


In [ ]:
df_clean['tweets'] = df_clean['tweets'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
df_clean.head()

In [ ]:
df_clean['tweets'][0]

In [ ]:
vec,x = tf_idf(df_clean['tweets'])
label = np.array(df_clean['label'].values)

#### SMOTE

In [ ]:
sampler = SMOTE()
x_so, label_so = sampler.fit_resample(x,label)

In [ ]:
Label_so = pd.DataFrame(data=label_so, columns=['label'])

In [ ]:
count_label = Label_so['label'].value_counts().to_frame()
px.bar(data_frame=count_label, x=count_label.index , y='label')

####Split Train-Test

In [ ]:
X_train_so, X_test_so, y_train_so, y_test_so = train_test_split(x_so, label_so, test_size=0.25, random_state=1)
print(X_train_so.shape)
print(X_test_so.shape)
print(y_train_so.shape)
print(y_test_so.shape)

In [ ]:
y_test_so[0]

In [ ]:
lb_make = LabelEncoder()
y_train_so = lb_make.fit_transform(y_train_so)
print(y_train_so.shape)

In [ ]:
y_test_so = lb_make.fit_transform(y_test_so)
print(y_test_so.shape)

In [ ]:
y_test_so[0]

### **Classification**


##### **Train**

In [ ]:
parameter_space = {
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
scoring = {'AUC': 'roc_auc', 
         'Accuracy': 'accuracy', 
         'Precision': 'precision_weighted', 
         'Recall':'recall_weighted', 
         'F1-Score':'f1_weighted'}


In [ ]:
k = StratifiedKFold(n_splits=10, shuffle=False)
clf = GridSearchCV(MLPClassifier(max_iter=100), parameter_space, scoring=scoring, n_jobs=-1, cv=k,refit='AUC')
start = time.time()
clf.fit(X_train_so.toarray(), y_train_so)
stop = time.time()
print('Waktu Train ANN Over-sampling: {:.2f} s'.format(stop - start))

In [ ]:
print("Accuracy Score:",clf.cv_results_['mean_test_Accuracy'][clf.best_index_])
print("Precision Score:",clf.cv_results_['mean_test_Precision'][clf.best_index_])
print("Recall Score:",clf.cv_results_['mean_test_Recall'][clf.best_index_])
print("F1-Score:",clf.cv_results_['mean_test_F1-Score'][clf.best_index_])
print("AUC Score:",clf.cv_results_['mean_test_AUC'][clf.best_index_])

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_estimator_)

##### **Test**

In [ ]:
y_pred_soann = clf.predict(X_test_so.toarray())
# y_pred_soann

##### **Evaluation**

In [ ]:
#Show the Confussion Matrix
cm1 = metrics.confusion_matrix(y_test_so, y_pred_soann)
plot_confusion_matrix(cm1, figsize=(5,5))
plt.show()

In [ ]:
print(classification_report(y_test_so, y_pred_soann))

In [ ]:
y_pred_soann_proba = clf.predict_proba(X_test_so.toarray())[::,1]
fprsoann, tprsoann, _ = metrics.roc_curve(y_test_so,  y_pred_soann_proba)
aucsoann = metrics.roc_auc_score(y_test_so, y_pred_soann_proba)

# ROC Curves
plt.plot(fprsoann,tprsoann,label="Multi Layer Perceptron, auc="+str(aucsoann))
plt.title('ROC Curve')
plt.xlabel('false positive rate') 
plt.ylabel('true positive rate')
plt.legend(loc=4)
plt.show()

##### **User Test**

In [ ]:
sentence = "lu" #@param
sentence = [sentence]
sentence = pd.DataFrame(data=sentence,columns=['text'], index=[0])
sentence['text'] = clean(sentence['text'])
sentence['text'] = case_fold(sentence['text'])
sentence['text'] = token(sentence['text'])
sentence['text'] = stop_words(sentence['text'])
sentence['text'] = stem(sentence['text'])
sentence['text'] = sentence['text'].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
clean_sentence = vec.transform(sentence['text'])
y_pred = clf.predict(clean_sentence.toarray())
label_pred = 'Rasis' if np.round(y_pred[0]) else 'Tidak Rasis'
print('predicted =',label_pred)